In [1129]:
import pandas as pd
import numpy as np
import string
import pickle
import itertools
from collections import Counter
import unidecode
import re
pd.set_option('max_colwidth',800)

## Published comments

In [1130]:
# FR: 24H + TG files
TG_24_pos = pd.read_csv("data/52_54_published.txt", sep='\t', encoding = "latin1", error_bad_lines=False)

# Matin files
matin_pos = pd.read_csv("data/53_published_all.tsv", sep='\t', error_bad_lines=False)

# Merge all published comments
my_pos = pd.concat([TG_24_pos, matin_pos])

print(my_pos["customer_id_plattform"].value_counts().sort_index())

#52 = TG
#53 = Matin
#54 = 24H
      

b'Skipping line 9576: expected 5 fields, saw 7\nSkipping line 10031: expected 5 fields, saw 7\nSkipping line 17089: expected 5 fields, saw 7\nSkipping line 18863: expected 5 fields, saw 7\nSkipping line 21292: expected 5 fields, saw 7\nSkipping line 24161: expected 5 fields, saw 7\nSkipping line 41983: expected 5 fields, saw 7\nSkipping line 43763: expected 5 fields, saw 7\nSkipping line 59074: expected 5 fields, saw 7\nSkipping line 61347: expected 5 fields, saw 7\nSkipping line 80300: expected 5 fields, saw 7\nSkipping line 83727: expected 5 fields, saw 7\nSkipping line 85462: expected 5 fields, saw 7\nSkipping line 86236: expected 5 fields, saw 7\nSkipping line 115688: expected 5 fields, saw 7\nSkipping line 115700: expected 5 fields, saw 7\nSkipping line 121512: expected 5 fields, saw 7\nSkipping line 122309: expected 5 fields, saw 7\nSkipping line 122950: expected 5 fields, saw 7\n'
b'Skipping line 143533: expected 5 fields, saw 7\nSkipping line 149948: expected 5 fields, saw 7\nS

52    167762
53    799579
54     50690
Name: customer_id_plattform, dtype: int64


## Unpublished comments

In [1131]:
vq_neg = pd.read_csv("data/54_rejected_all.tsv", sep='\t', encoding = "latin1", error_bad_lines=False)
tg_neg = pd.read_csv("data/52_rejected_all.tsv", sep='\t', encoding = "latin1", error_bad_lines=False)
matin_neg = pd.read_csv("data/53_rejected_all.tsv", sep='\t', error_bad_lines=False)

my_neg = pd.concat([tg_neg, vq_neg, matin_neg])

print(my_neg["customer_id_plattform"].value_counts().sort_index())

b'Skipping line 14992: expected 5 fields, saw 7\nSkipping line 26132: expected 5 fields, saw 7\nSkipping line 30231: expected 5 fields, saw 7\nSkipping line 30620: expected 5 fields, saw 7\n'
b'Skipping line 358: expected 5 fields, saw 7\nSkipping line 4689: expected 5 fields, saw 7\nSkipping line 15581: expected 5 fields, saw 7\nSkipping line 18657: expected 5 fields, saw 7\nSkipping line 25984: expected 5 fields, saw 7\nSkipping line 32551: expected 5 fields, saw 7\nSkipping line 34266: expected 5 fields, saw 7\nSkipping line 34310: expected 5 fields, saw 7\nSkipping line 34366: expected 5 fields, saw 7\nSkipping line 34514: expected 5 fields, saw 7\nSkipping line 34548: expected 5 fields, saw 7\nSkipping line 48123: expected 5 fields, saw 7\nSkipping line 110698: expected 5 fields, saw 7\nSkipping line 129997: expected 5 fields, saw 7\n'
b'Skipping line 135431: expected 5 fields, saw 9\nSkipping line 210725: expected 5 fields, saw 7\nSkipping line 211875: expected 5 fields, saw 7\nS

52     56883
53    247206
54     17759
Name: customer_id_plattform, dtype: int64


In [1132]:
my_pos = my_pos[my_pos["moderation"] == "OK"]
my_neg = my_neg[my_neg['moderation']== "NOK"]

In [1133]:
# Published vs unpublished rate
print(len(my_neg)/(len(my_pos)+len(my_neg)))

0.22727645418090772


## Checking for Nans

In [1134]:
print(len(my_pos))
my_pos_no_nan = my_pos[~my_pos['message'].isnull()]
len(my_pos_no_nan)

1017833


1017833

In [1135]:
print(len(my_neg))
my_neg = my_neg[~my_neg['message'].isnull()]
len(my_neg)

299369


299334

## Testing a string

In [1136]:
# Maurer in published comments
len(my_pos[my_pos['message'].str.contains("Maurer")]['message'])

1243

In [1137]:
#Maurer in unpublished comments
len(my_neg[my_neg['message'].str.contains("Maurer")]['message'])

225

In [1138]:
252/1243

0.20273531777956558

In [1139]:
#Sommaruga in published comments
len(my_pos[my_pos['message'].str.contains("Sommaruga")]['message'])

2270

In [1140]:
#Sommaruga in unpublished comments
len(my_neg[my_neg['message'].str.contains("Sommaruga")]['message'])

658

In [1141]:
676/2270

0.29779735682819386

# Measure the Carlo Sommaruga effet

In [1142]:
my_pos.reset_index(drop=True, inplace=True)
my_neg.reset_index(drop=True, inplace=True)

In [1143]:
# function to identify comments talking about Carlo or Simonetta

def isCarlo(message):
    
    cherchez = {
        "C": {
            "terms": ["Monsieur Sommaruga", "monsieur Sommaruga", "M\. Sommaruga", "m\. Sommaruga","C\. Sommaruga", "M Sommaruga", "Carlo", "CN Sommaruga", "gauche genevoise", "Salermo", "Maudet", "Tornare", "un socialiste", "avocat"],
            "counter": 0
        },
        "S": {
            "terms": ["Madame Sommaruga", "madame Sommaruga", "Mrs Sommaruga","Mme Sommaruga", "mme Sommaruga","Mme\. Sommaruga", "la Sommaruga", "La Sommaruga", "dame Sommaruga", "maman Sommaruga", "Somaruga","S\. Sommaruga", "S\.Sommaruga", "Simonetta", "Simmonnetta", "Simmonetta", "Simonneta", "Simonnetta","CF Sommaruga", "Maurer", "Burkhalter", "Leuthard", "Berset", "Cassis", "Juncker", "Merkel", "Macron", "Trump", "CF", "conseillère", "Europe", "une socialiste", "réfugié", "réfugiés", "Ministre", "ministre"],
            "counter": 0
        }
    }
    
    for who in cherchez:
        for term in cherchez[who]["terms"]:
            if re.search(term, message) != None:
                cherchez[who]["counter"] += len(tuple(re.finditer(term, message)))
                
                
    if cherchez["C"]["counter"] > cherchez["S"]["counter"]:
   
        return True

    elif cherchez["C"]["counter"] < cherchez["S"]["counter"]:
       
        return False
    else:
        
        return None

In [1144]:
isCarlo("de l'élection d'Alain Berset ou de Simonetta Sommaruga.")

False

In [1145]:
sommaruga = my_pos[my_pos['message'].str.contains("Sommaruga")]
sommaruga_neg = my_neg[my_neg['message'].str.contains("Sommaruga")]

In [1146]:
sommaruga['isCarlo'] = sommaruga['message'].apply(isCarlo)
len(sommaruga[sommaruga['isCarlo'] == True])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


105

In [1147]:
sommaruga_neg['isCarlo'] = sommaruga_neg['message'].apply(isCarlo)
len(sommaruga_neg[sommaruga_neg['isCarlo'] == True])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


67

In [1148]:
# Add new variable

carlo = sommaruga[sommaruga['isCarlo'] == True]
carlo_neg = sommaruga_neg[sommaruga_neg['isCarlo'] == True]

In [1149]:
# Merge back with main dfs

my_pos = my_pos.merge(carlo[['isCarlo']], how="left", left_index=True, right_index=True)
my_neg = my_neg.merge(carlo_neg[['isCarlo']], how="left", left_index=True, right_index=True)

In [1150]:
my_pos['isCarlo'].value_counts()

True    105
Name: isCarlo, dtype: int64

## Dealing with French accents and punctuation

In [1151]:
my_strings = "L'Allemagne, François, vous-mêmes, Widmer-Schlumpf, Schneider-Ammann et Marie-Ange;!"


In [1152]:
# transform accents into naked letters
unaccented_string = unidecode.unidecode(my_strings)
unaccented_string

"L'Allemagne, Francois, vous-memes, Widmer-Schlumpf, Schneider-Ammann et Marie-Ange;!"

In [1153]:
# Custom homemade punctuation removal for French 

def french_strip(my_string):
    
    #my_string = my_string.lower()
    
    my_string = unidecode.unidecode(my_string)

    punct_list = [",","!","#","$","%","&","(",")","*","+","-",".","/",":",";","<","=",">","?","[","]","^","_","`","{","|","}","~"]

    for punct in punct_list:
        my_string = my_string.replace(punct, "")

    my_string = my_string.replace("'", " ")
    

    return(my_string)

In [1154]:
french_strip(my_strings)

'L Allemagne Francois vousmemes WidmerSchlumpf SchneiderAmmann et MarieAnge'

In [1155]:
# Exclude IsCarlo Sommaruga

In [1156]:
my_pos.head()

,customer_id_plattform,datetime,message,moderation,storyid,isCarlo
0,52,2016-01-22 12:34:29,"Pourquoi ne pas construire des logements à Meyrin, cette jolie commune qui a déjà fait ses preuves dans le domaine de la construction plus que correcte d'immeubles à des prix abordables, avec zone verte , et laisser la caserne aux Vernets ? C'est un non-sens et de l'argent gaspillé. Comme dit , ce n'est pas ""catholique"" (ni protestant, ni agnostique - mais cette remarque n'est qu'humoristique) !",OK,26858451,NaN
1,52,2016-01-22 11:44:54,Welcome Back!! Sa mentalité exemplaire va faire du bien dans le vestiaire!!,OK,25085337,NaN
2,52,2016-01-22 11:44:24,"Si le deuxième tour des élections c'est Hollande vs Sarko, alors se sera officiel : l'opinion publique française aurait complètement perdu la tête !",OK,27535106,NaN
3,52,2016-01-22 11:40:10,"Tout à fait Micka, tout ce beau monde est vraiment stupéfiant.",OK,24030450,NaN
4,52,2016-01-22 11:37:08,Bravos Roger pour le tennis et le pays,OK,20203928,NaN


In [1157]:
pos_cm = my_pos[my_pos['isCarlo'] != True]["message"]
len(pos_cm)

1017728

In [1158]:
neg_cm = my_neg[my_neg['isCarlo'] != True]['message']
len(neg_cm)

299267

In [1159]:
# Remove punctuation (! long inefficient method that still works)
# Barney method for German: translator = str.maketrans('','',string.punctuation)


pos_cm_final = [french_strip(x) for x in pos_cm]

In [1160]:
neg_cm_final = [french_strip(x) for x in neg_cm]

## Calculate word frequencies

In [1161]:
i = 0
for m in pos_cm_final:
    if m.find("maurer") != -1:
        i+=1
print(i)
print(str(i / len(pos_cm_final) * 100) + ("%"))

43
0.004225097472016099%


In [1162]:
# Preprocess

# Number of times a word appear
pos_cm_counter = Counter(itertools.chain(*[x.split() for x in pos_cm_final]))
neg_cm_counter = Counter(itertools.chain(*[x.split() for x in neg_cm_final]))


# Total of all words
pos_cm_sum = sum(pos_cm_counter.values())
neg_cm_sum = sum(neg_cm_counter.values())


pos_cm_counter_clean = dict()
neg_cm_counter_clean = dict()

# If the word appears more than 50 times, divide the appearance number by the sum of all words
for k,v in pos_cm_counter.items():
    if v > 20:
        pos_cm_counter_clean[k] = v/pos_cm_sum

for k,v in neg_cm_counter.items():
    if v > 20:
        neg_cm_counter_clean[k] = v/neg_cm_sum

In [1163]:
# Final results
neg_pos_term = dict()

In [1164]:
# list of unique words that are keys in both dictionaries

inters = set(pos_cm_counter_clean.keys()).intersection(set(neg_cm_counter_clean.keys()))

In [1165]:
# for each of these unique words, divide the ratio appearance/sum of neg by the ratio appearance/sum of pos

for t in inters:
    
    neg_pos_term[t] = neg_cm_counter_clean[t] / pos_cm_counter_clean[t]

In [1166]:
# Creating Dataframe
df = pd.DataFrame(list(neg_pos_term.items()))

In [1167]:
# Renaming
df.columns = ['Word', 'Value']

In [1168]:
# Deleting words shorter than 2 digits

def length(elem):
    return len(list(elem))

df['Character Count'] = df['Word'].apply(length)

df = df[df['Character Count'] > 1]
len(df)

19832

## -----Analysis-----

In [1169]:
df.sort_values(by='Value', ascending=False)[:10]

,Word,Value,Character Count
19860,abruti,37.259844,6
7613,trolls,34.364969,6
14024,troll,33.657224,5
15865,httpluttepourlechangementoverblogcomClairement,30.175689,46
8634,mahometans,29.480652,10
19147,con,25.781862,3
9032,ordure,24.547925,6
18274,guignol,24.431758,7
18649,abrutis,24.338577,7
18943,barbues,23.964587,7


## Federal council

In [1170]:
test = my_pos[my_pos['message'].str.contains("Pierre-André Page")]
test

,customer_id_plattform,datetime,message,moderation,storyid,isCarlo
701442,53,2015-10-17 18:10:19,"C'est faux Nicole. La gauche a toujours été anti-police et anti-militaire. Il semblerait que vous l'ayez oublié. L'UDC a maintes fois demandé plus d'effectifs. \Un signal d’alarme qui a été répercuté par deux députés UDC, Nicolas Kolly (Essert) et Pierre-André Page (Châtonnaye), dépositaires d’une motion.\""""",OK,21304865,NaN


In [1171]:
bundesräte = ['Sommaruga', 'Maurer', "Burkhalter",
              'Cassis', 'Leuthard', 'Parmelin',
              'SchneiderAmmann', 'Ammann', 'Berset', "WidmerSchlumpf", 'EWS']
nl = []

for elem in bundesräte:
    print(df[df['Word'] == elem]['Value'].values)
    mini_dict = {'Country': elem,
                 'Value': df[df['Word'] == elem]['Value'].values}
    nl.append(mini_dict)
nl

[0.90321143]
[0.61083097]
[0.62273684]
[0.38293469]
[0.45501919]
[0.37098665]
[0.46085744]
[]
[0.61763455]
[0.52294518]
[0.43500447]


[{'Country': 'Sommaruga', 'Value': array([0.90321143])},
 {'Country': 'Maurer', 'Value': array([0.61083097])},
 {'Country': 'Burkhalter', 'Value': array([0.62273684])},
 {'Country': 'Cassis', 'Value': array([0.38293469])},
 {'Country': 'Leuthard', 'Value': array([0.45501919])},
 {'Country': 'Parmelin', 'Value': array([0.37098665])},
 {'Country': 'SchneiderAmmann', 'Value': array([0.46085744])},
 {'Country': 'Ammann', 'Value': array([], dtype=float64)},
 {'Country': 'Berset', 'Value': array([0.61763455])},
 {'Country': 'WidmerSchlumpf', 'Value': array([0.52294518])},
 {'Country': 'EWS', 'Value': array([0.43500447])}]

In [1172]:
fl = []
for elem in nl:
    if len(elem['Value']) < 1:
        continue
    else:
        new = elem['Value'][0]
        mini_dict = {'Country':elem['Country'],
                     'Value':new}
        fl.append(mini_dict)
pd.DataFrame(fl).sort_values(by='Value', ascending=False)

,Country,Value
0,Sommaruga,0.903211
2,Burkhalter,0.622737
7,Berset,0.617635
1,Maurer,0.610831
8,WidmerSchlumpf,0.522945
6,SchneiderAmmann,0.460857
4,Leuthard,0.455019
9,EWS,0.435004
3,Cassis,0.382935
5,Parmelin,0.370987


## National council, State council, VD + GE cantonal party presidents, Lausanne & Geneva executive council members

In [1173]:
pol = pd.read_csv('data/pol_fr.csv')
len(pol)

240

In [1174]:
def lastname(elem):
    elem = elem.split(' ')[0]
    if '-' in elem:
        return elem.split('-')[1]
    else: 
        return elem
pol['Lastname'] = pol['Nationalrat'].apply(lastname)
lst = list(pol['Lastname'])

nl = []
for elem in lst:
    mini_dict = {'Country': elem,
                 'Value': df[df['Word'] == elem]['Value'].values}
    nl.append(mini_dict)
fl = []
for elem in nl:
    if len(elem['Value']) < 1:
        continue
    else:
        new = elem['Value'][0]
        mini_dict = {'Country':elem['Country'],
                     'Value':new}
        fl.append(mini_dict)
pd.DataFrame(fl).sort_values(by='Value', ascending=False).drop_duplicates()

,Country,Value
5,Page,4.847675
44,Tosato,4.403749
0,Giezendanner,2.297608
3,Graf,1.923156
30,Moser,1.631185
20,Walter,1.627472
43,Roch,1.585350
10,Golay,1.495839
22,Arnold,1.403695
35,Maillard,1.377155


## Parteien-Rating

In [1175]:
p = ['PS', 'PLR', 'UDC', 'Verts', 'Vert li\'b', 'PDC', 'Pop', 'Solidarités', "Ensemble à Gauche"]
nl = []
for elem in p:
    mini_dict = {'Country': elem,
                 'Value': df[df['Word'] == elem]['Value'].values}
    nl.append(mini_dict)
fl = []
for elem in nl:
    if len(elem['Value']) < 1:
        continue
    else:
        new = elem['Value'][0]
        mini_dict = {'Country':elem['Country'],
                     'Value':new}
        fl.append(mini_dict)
pd.DataFrame(fl).sort_values(by='Value', ascending=False)

,Country,Value
2,UDC,0.814293
0,PS,0.795412
4,PDC,0.616553
3,Verts,0.436036
1,PLR,0.420849


## State leader

In [1176]:
p = ['Trump', 'Erdogan', 'May', 'Macron', 'Putin', 'Merkel', 'Juncker', 'Hillary']
nl = []
for elem in p:
    mini_dict = {'Country': elem,
                 'Value': df[df['Word'] == elem]['Value'].values}
    nl.append(mini_dict)
fl = []
for elem in nl:
    if len(elem['Value']) < 1:
        continue
    else:
        new = elem['Value'][0]
        mini_dict = {'Country':elem['Country'],
                     'Value':new}
        fl.append(mini_dict)
pd.DataFrame(fl).sort_values(by='Value', ascending=False)

,Country,Value
4,Putin,3.377876
1,Erdogan,1.366614
7,Hillary,1.077004
5,Merkel,0.999282
0,Trump,0.840149
6,Juncker,0.771982
3,Macron,0.673952
2,May,0.324276


## Resample

In [1177]:
my_pos['date_custom'] = pd.to_datetime(my_pos['datetime'])
my_neg['date_custom'] = pd.to_datetime(my_neg['datetime'])

In [1178]:
my_pos.set_index(my_pos['date_custom'], inplace=True)
my_neg.set_index(my_neg['date_custom'], inplace=True)

In [1179]:
print(len(my_pos["2014":"2017"]))
len(my_neg["2014":"2017"])

999943


295369

# By year

In [1180]:
my_pos.resample('A').count()

,customer_id_plattform,datetime,message,moderation,storyid,isCarlo,date_custom
date_custom,,,,,,,
2014-12-31,261988,261988,261988,261988,261988,27,261988
2015-12-31,368219,368219,368219,368219,368219,44,368219
2016-12-31,251565,251565,251565,251565,251565,23,251565
2017-12-31,118171,118171,118171,118171,118171,11,118171
2018-12-31,17890,17890,17890,17890,17890,0,17890


In [1181]:
my_neg.resample('A').count()

,storyid,customer_id_plattform,datetime,moderation,message,isCarlo,date_custom
date_custom,,,,,,,
2014-12-31,87759,87759,87759,87759,87759,29,87759
2015-12-31,108206,108206,108206,108206,108206,33,108206
2016-12-31,71684,71684,71684,71684,71684,4,71684
2017-12-31,27720,27720,27720,27720,27720,1,27720
2018-12-31,3965,3965,3965,3965,3965,0,3965


In [1182]:
my_pos.groupby('customer_id_plattform').resample('A').count()

customer_id_plattform  datetime  message  \
customer_id_plattform date_custom                                             
52                    2014-12-31                     151       151      151   
                      2015-12-31                   64280     64280    64280   
                      2016-12-31                   56026     56026    56026   
                      2017-12-31                   42217     42217    42217   
                      2018-12-31                    5048      5048     5048   
53                    2014-12-31                  261762    261762   261762   
                      2015-12-31                  285917    285917   285917   
                      2016-12-31                  180156    180156   180156   
                      2017-12-31                   60957     60957    60957   
                      2018-12-31                   10638     10638    10638   
54                    2014-12-31                      75        75       75   
                      2015-12-31                   18022     18022    18022   
                      2016-12-31                   15383     15383    15383   
                      2017-12-31                   14997     14997    14997   
                      2018-12-31                    2204      2204     2204   

                                   moderation  storyid  isCarlo  date_custom  
customer_id_plattform date_custom                                             
52                    2014-12-31          151      151        0          151  
                      2015-12-31        64280    64280       21        64280  
                      2016-12-31        56026    56026       13        56026  
                      2017-12-31        42217    42217        4        42217  
                      2018-12-31         5048     5048        0         5048  
53                    2014-12-31       261762   261762       27       261762  
                      2015-12-31       285917   285917       20       285917  
                      2016-12-31       180156   180156        6       180156  
                      2017-12-31        60957    60957        5        60957  
                      2018-12-31        10638    10638        0        10638  
54                    2014-12-31           75       75        0           75  
                      2015-12-31        18022    18022        3        18022  
                      2016-12-31        15383    15383        4        15383  
                      2017-12-31        14997    14997        2        14997  
                      2018-12-31         2204     2204        0         2204

In [1183]:
my_pos.resample('M').count()

,customer_id_plattform,datetime,message,moderation,storyid,isCarlo,date_custom
date_custom,,,,,,,
2014-01-31,20263,20263,20263,20263,20263,2,20263
2014-02-28,22356,22356,22356,22356,22356,1,22356
2014-03-31,21079,21079,21079,21079,21079,3,21079
2014-04-30,17485,17485,17485,17485,17485,4,17485
2014-05-31,20114,20114,20114,20114,20114,4,20114
2014-06-30,21535,21535,21535,21535,21535,0,21535
2014-07-31,26205,26205,26205,26205,26205,3,26205
2014-08-31,23452,23452,23452,23452,23452,0,23452
2014-09-30,22505,22505,22505,22505,22505,6,22505


In [1184]:
# ! Modification de la charte en 2016
# ! Paywall en 2015
# --> No annual average available

## Additional Text analysis

In [1185]:
print("Positive Count:", str(len(pos_cm)))
print("Negative Count:", str(len(neg_cm)))
print("Total:", str(len(pos_cm) + len(neg_cm)))

Positive Count: 1017728
Negative Count: 299267
Total: 1316995


In [1186]:
len(my_neg[my_neg['message'].str.contains("Maurer")]['message'])

225

In [1187]:
maurer = my_neg[my_neg['message'].str.contains("Maurer")]
maurer.resample('A').count()

,storyid,customer_id_plattform,datetime,moderation,message,isCarlo,date_custom
date_custom,,,,,,,
2014-12-31,99,99,99,99,99,0,99
2015-12-31,65,65,65,65,65,0,65
2016-12-31,37,37,37,37,37,0,37
2017-12-31,16,16,16,16,16,0,16
2018-12-31,8,8,8,8,8,0,8
